In [1]:
import pandas as pd

# read the dataset into a pandas dataframe object

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)

#print some data to make sure the data was properly read into the dataframe

In [3]:
print(df.head(n=3))
rows,columns = df.shape


         0  1      2      3      4       5        6        7       8   \
0    842302  M  17.99  10.38  122.8  1001.0  0.11840  0.27760  0.3001   
1    842517  M  20.57  17.77  132.9  1326.0  0.08474  0.07864  0.0869   
2  84300903  M  19.69  21.25  130.0  1203.0  0.10960  0.15990  0.1974   

        9    ...        22     23     24      25      26      27      28  \
0  0.14710   ...     25.38  17.33  184.6  2019.0  0.1622  0.6656  0.7119   
1  0.07017   ...     24.99  23.41  158.8  1956.0  0.1238  0.1866  0.2416   
2  0.12790   ...     23.57  25.53  152.5  1709.0  0.1444  0.4245  0.4504   

       29      30       31  
0  0.2654  0.4601  0.11890  
1  0.1860  0.2750  0.08902  
2  0.2430  0.3613  0.08758  

[3 rows x 32 columns]


# get a sense of the size of the dataset

In [4]:
print('# of rows:', rows)
print('# of columns:', columns)


('# of rows:', 569)
('# of columns:', 32)


# first column is just an ID number, second column is the diagnostics - M for Malignant and B for Benign

# 3rd to 32nd column include the 30 features of the dataset

# assign the 30 features of the dataset and the target variable to separate Numpy arrays

# same result in the next 2 lines if we replace iloc with loc

In [1]:
X = df.iloc[:, 2:].values
Y = df.iloc[:, 1].values

NameError: name 'df' is not defined

# print X and Y by typing the variable names in console to make sure you properly extrated the
# independent and dependent variables. You can also directly look at the contents of these variables
# using the variabe explorer on the right
# transform the class labels from their original string representation (M and B) to integers

In [7]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
Y = le.fit_transform(Y)

# now Malignant is 1 and Benign is 0
# divide the dataset into 80% training and 20% separate test data

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

/Users/Pramod/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# create a pipeline with the following steps chained to each other
# 1. for optimal performance transform all feature values into the same scale - standardize the columns
# before feeding them to the classifier
# 2. Compress the initial 30 dimensional data into a lower 2 dimensional space using PCA
# 3. Apply the logistic regression algorithm


In [10]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline


# Define the steps of the pipeline, each step is a tuple: a name and either a transformer or estimator object


In [11]:
classification_pipeline = Pipeline([('standard_scaler', StandardScaler()),
                                    ('pca', PCA(n_components=2)),
                                    ('classifier',LogisticRegression(random_state=1))])

# fit the training data into the model

In [12]:
classification_pipeline.fit(X_train,Y_train)
# compute the accuracy of the model on test data 
accuracy = classification_pipeline.score(X_test, Y_test) 
print('Test Accuracy: %.3f' % accuracy)

Test Accuracy: 0.947


# instead of just a single test on test data, let's do a stratified k-fold cross validation on training data

In [13]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(estimator=classification_pipeline, X=X_train, y=Y_train, cv=10, n_jobs=1) 
import numpy as np
print('CV accuracy %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy 0.950 +/- 0.029


# let's take a look at the confusion matrix

In [14]:
from sklearn.metrics import confusion_matrix
y_pred = classification_pipeline.predict(X_test)
confmat = confusion_matrix(y_true=Y_test,y_pred=y_pred)
print(confmat)

[[71  1]
 [ 5 37]]


# we get [[71,1][5,32]] first list is for class 0 (-) and second list is for class 1 (+)
# to convert this into my convention (a) stack the 2 lists, one in each row to get # [71, 1] [TN, FP] rows are TRUE CLASS, columns are PREDICTED class
# [5, 32] [FN, TP]
# Then simply switch the elements on the diagonal to get
# [31, 1] [TP, FP] rows are PREDICTED CLASS, columns are TRUE CLASS
# [5, 71] [FN, TN]


In [15]:
from sklearn.metrics import classification_report
print('****************:', classification_report(y_true=Y_test,y_pred=y_pred))


('****************:', '             precision    recall  f1-score   support\n\n          0       0.93      0.99      0.96        72\n          1       0.97      0.88      0.93        42\n\navg / total       0.95      0.95      0.95       114\n')


# compute precision, recall, and F measure
# we have already seen this as part of the classification report printed in the above line 
# classification report gives these masures for both the positive and negative class
# what we get here is only for the positive class

In [16]:
from sklearn.metrics import precision_score
print('Precision: %.3f' % precision_score(y_true=Y_test, y_pred=y_pred))
from sklearn.metrics import recall_score, f1_score
print('Recall: %.3f' % recall_score(y_true=Y_test, y_pred=y_pred)) 
print('F1: %.3f' % f1_score(y_true=Y_test, y_pred=y_pred))


Precision: 0.974
Recall: 0.881
F1: 0.925


# now let's try the logistic regression model with different parameters and try
# to finetune these hyperpatameters using the Grid Search approach
# Any parameter provided when constructing an estimator may be optimized in this manner.
# Specifically, to find the names and current values for all parameters for a given estimator, use: 
# estimator.get_params()

In [17]:
from sklearn.grid_search import GridSearchCV
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# note that a list of available parameters to tune can be displayed by typing this in the console: 
# classification_pipeline.get_params().keys()


print('Paramters available for tuning:', classification_pipeline.get_params().keys())


# also note that the param name is constructed as the string name given to it
# in the pipeline constructor followed by 2 underscores and then the param name

param_grid = [{'classifier__penalty':['l1', 'l2'], 'classifier__C': param_range}]


# note that in the above line each set of items enclosed in {} defines one grid
# here we only have one grid, but we could have multiple grids to explore


gs = GridSearchCV(estimator=classification_pipeline,
          param_grid=param_grid, scoring= 'accuracy', cv=10,
          n_jobs=-1)

gs = gs.fit(X_train, Y_train)
print('Best accuracy score:',gs.best_score_) 
print('Best Parameters:',gs.best_params_)

('Paramters available for tuning:', ['standard_scaler__copy', 'pca__whiten', 'classifier__max_iter', 'classifier__C', 'classifier__multi_class', 'standard_scaler__with_mean', 'pca__iterated_power', 'classifier__intercept_scaling', 'classifier__warm_start', 'pca', 'pca__random_state', 'classifier__class_weight', 'pca__tol', 'pca__copy', 'pca__svd_solver', 'classifier__solver', 'classifier__dual', 'classifier__fit_intercept', 'classifier__n_jobs', 'pca__n_components', 'classifier__tol', 'standard_scaler__with_std', 'standard_scaler', 'steps', 'classifier__random_state', 'classifier', 'classifier__verbose', 'classifier__penalty'])


/Users/Pramod/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


('Best accuracy score:', 0.9516483516483516)
('Best Parameters:', {'classifier__C': 1.0, 'classifier__penalty': 'l1'})
